In [1]:
def install_packages():
    # パッケージopen3d、laspy、pygltflibのインストール
    !pip install open3d laspy pygltflib trimesh

def create_directories():
    # ディレクトリを作成
    !mkdir -p target result mesh

def add_points(points, colors, las):
    scales = las.header.scales
    points = np.vstack([points, np.column_stack([las.points.X * scales[0], las.points.Y * scales[1], las.points.Z * scales[2]])])
    if hasattr(las.points, 'red'):
        colors = np.vstack([colors, np.column_stack([las.points.red, las.points.green, las.points.blue]) / 65535.])  # カラー
    else:
        colors = np.vstack([colors, np.column_stack([np.full((len(las.X), 3), 0.5)])])  # グレースケール
    return points, colors

def load_files(files):
    vec = np.empty((0, 3))
    col = np.empty((0, 3))

    for filename in files:
        las = laspy.read(filename)
        vec, col = add_points(vec, col, las)
        print(f"{filename} => {len(vec)} points (total)")

    # 端が原点となるように移動
    min_val = np.amin(vec, axis=0)
    max_val = np.amax(vec, axis=0)
    vec = vec - min_val
    bbox = max_val - min_val
    print(f"size: {bbox[0]:.1f} x {bbox[1]:.1f} x {bbox[2]:.1f} (m)")
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(vec)
    pcd.colors = o3d.utility.Vector3dVector(col)
    return pcd

def create_mesh(point_cloud, mesh_depth):
    # 法線の推定
    point_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=20))
    point_cloud.orient_normals_to_align_with_direction(orientation_reference=np.array([0., 0., 1.]))  # Upの軸を指定する

    # メッシュ化
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        poisson_mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(point_cloud, depth=mesh_depth)
    print(poisson_mesh)

    bbox = point_cloud.get_axis_aligned_bounding_box()
    mesh = poisson_mesh.crop(bbox)

       return mesh

def write_mesh(filename, mesh):
    body, ext = os.path.splitext(filename)
    if ext == ".glb":
        gltf_file = body + ".gltf"
        o3d.io.write_triangle_mesh(gltf_file, mesh, write_ascii=False, write_vertex_normals=True)
        gltf2glb(gltf_file, override=True)
        os.remove(gltf_file)
    else:
        o3d.io.write_triangle_mesh(filename, mesh, write_ascii=False, write_vertex_normals=True)

def mesh_conversion_and_save(pcd, output_folder, mesh_depth=10):
    mesh = create_mesh(pcd, mesh_depth)
    output_path = os.path.join(output_folder, 'output.ply')  # 出力ファイルのパス

    # メッシュの保存
    write_mesh(output_path, mesh)

    return output_path

def convert_ply_to_obj(input_ply, output_obj):
    mesh = trimesh.load_mesh(input_ply)  # PLYファイルを読み込む

    # OBJファイルに変換して保存
    mesh.export(output_obj, file_type='obj')

def main():
    install_packages()
    create_directories()

    folder_path = '/content/target'  # フォルダのパスを指定
    files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.las')]

    pcd = load_files(files)

    # メッシュに変換して保存
    output_mesh_path = mesh_conversion_and_save(pcd, mesh_folder)
    print(f"Mesh saved to: {output_mesh_path}")

    # PLYをOBJに変換して保存
    convert_ply_to_obj(input_ply_path, output_obj_path)
    print(f"Converted to OBJ: {output_obj_path}")

In [ ]:
#!pip install open3d laspy pygltflib trimesh
install_packages()

import os
import numpy as np
import open3d as o3d
import laspy
import trimesh
from pygltflib.utils import gltf2glb

In [3]:
create_directories()

# targetフォルダにlasファイルをアップロードする
# アップロード完了後に次のセルを実行する

In [ ]:
if __name__ == "__main__":
    install_packages()

    folder_path = '/content/target'  # フォルダのパスを指定
    files = [os.path.join(folder_path, filename) for filename in os.listdir(folder_path) if filename.endswith('.las')]

    pcd = load_files(files)

    # メッシュに変換して保存
    mesh_folder = '/content/mesh' # フォルダのパスを指定
    output_mesh_path = mesh_conversion_and_save(pcd, mesh_folder)
    print(f"Mesh saved to: {output_mesh_path}")

    # PLYをOBJに変換して保存
    input_ply_path = '/content/mesh/output.ply' # フォルダのパスを指定
    output_obj_path = '/content/result/output.obj' # フォルダのパスを指定
    convert_ply_to_obj(input_ply_path, output_obj_path)
    print(f"Converted to OBJ: {output_obj_path}")

# resultフォルダに作成されたobjファイルをダウンロードする